In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql = '''
select o.order_number,od.product_id,od.product_name,o.merchant_id,o.merchant_name,o.`status`,o.create_time,op.pay_date,ol.go_express_date,o.alipay_order_id, tos.reality_refund_date 
,tos.order_id
,case o.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.status') end,'"','') as status_p 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.is_vip') end,'"','') as is_vip
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
from db_digua_business.t_order as o 
left join db_digua_business.t_order_pay as op on op.order_id = o.id and op.pay_type = 'ZFBYSQ' and op.sync_mini_order = 'Y'
left join db_digua_business.t_order_logistics as ol on o.id = ol.order_id 
left join db_digua_business.t_order_details as od on od.order_id = o.id
left join db_digua_business.t_order_stages as tos on tos.order_id=o.id
left join db_digua_business.t_order_risk tor on o.id = tor.order_id
left join db_digua_business.t_order_details tod on tod.order_id=o.id
left join db_digua_business.t_product tp on tp.id=tod.product_id
-- 渠道名称
left join db_digua_business.t_channel cc on o.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on o.activity_id = pa.id
-- left join db_digua_business.t_product_classify tpc on tpc.id=tp.classify_id
where o.create_time >= '2024-10-01 00:00:00'  and op.`status` in (2,5) and tp.classify_id in (116, 165) and o.order_method=1 and pa.type=2
and o.has_order_check = 'Y'
'''
df = query(sql)

In [4]:
date1 = '2024-11-15 00:00:00'
date2 = '2024-12-15 23:59:59'
sql2 = f'''
SELECT o.create_time, o.order_number,o.channel,c.`name`,o.merchant_name,o.`status`,o.cancel_reason,JSON_EXTRACT(ori.decision_result, "$.rejected") as rejected,JSON_EXTRACT(ori.decision_result, "$.status") as risk_status,order_method,o.has_order_check ,ol.go_express_date, tpr.min_create_time
from db_digua_business.t_order as o
left join db_digua_business.t_order_risk as ori on o.id = ori.order_id  
left join db_digua_business.t_order_details as od on o.id = od.order_id
left join db_digua_business.t_platform_activity as pa on pa.id = o.activity_id
left join db_digua_business.t_channel as c on c.scene = o.channel 
left join db_digua_business.t_order_logistics as ol on o.id = ol.order_id 
left join (SELECT tpr.order_id, min(tpr.create_time) as min_create_time FROM db_digua_business.t_pay_record tpr GROUP BY tpr.order_id) tpr on tpr.order_id=o.id
where `day` >=90 AND pa.type = 2
and o.create_time >= '{date1}' 
and o.create_time <= '{date2}' 
-- and o.`status` not in (1,13) 
and o.has_order_check = 'Y'
'''
df2 = query(sql2)

In [5]:
# df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['机审通过'] = np.where((df['进件']=='进件')&(df['是否机审强拒']==0), 1, 0)

In [6]:

# df_date = df.groupby('order_id').agg(min_date=('reality_refund_date', 'min'))
# df_date = df.groupby('order_id').agg(min_date=('create_time', 'min'))
# df_48_date = pd.merge(df, df_date, on='order_id', how='left')
# df_48_date['实际支付时间'] = df_48_date['min_date'].dt.date
# df_48_date = df_48_date[(df_48_date['min_date']>='2024-10-19 00:00:00')&(df_48_date['min_date']<='2024-11-19 00:00:00')&(df_48_date['create_time'].notna())&(df_48_date['go_express_date'].notna())]
# df_48_date_less = df_48_date[df_48_date['go_express_date']-df_48_date['min_date']<=pd.Timedelta(hours=48)]
# order_id_count = df_48_date['order_id'].nunique()
# order_id_count
# order_id_count_less = df_48_date_less['order_id'].nunique()
# order_id_count_less

In [7]:
# t_date = datetime.now().strftime('%Y%m%d')
# df_48_date_group = df_48_date.groupby('实际支付时间').agg(order_id_count=('order_id', 'nunique'))
# df_48_date_less_group = df_48_date_less.groupby('实际支付时间').agg(order_id_less_count=('order_id', 'nunique'))
# df_48 = pd.merge(df_48_date_group, df_48_date_less_group, on='实际支付时间', how='inner')
# df_48['发货率'] = df_48['order_id_less_count']/df_48['order_id_count']
# df_48['发货率'] = df_48['发货率'].apply(lambda x: format(x, '.2%'))
# df_48.rename(columns={'order_id_count': '支付时间', 'order_id_less_count': '48小时发货数'}, inplace=True)
# df_48.to_excel(f'F:/需求/瑞彤需求/48小时发货率_{t_date}.xlsx')
# df_48

In [8]:
df_48_date = df[(df['create_time']>='2024-10-19 00:00:00')&(df['create_time']<='2024-11-19 00:00:00')&(df['go_express_date'].notna())]
# df_48_date['diff'] = df_48_date['go_express_date']-df_48_date['create_time']
# df_48_date['diff'] = df_48_date['diff'].dt.total_seconds() / 3600
# df_48_date[df_48_date['diff']<=48][['order_id', 'order_number', 'create_time', 'go_express_date', 'diff', 'status_p']].drop_duplicates()

In [9]:
df_48_date[df_48_date['go_express_date']-df_48_date['create_time']<=pd.Timedelta(hours=48)]['create_time'].nunique()

487

In [10]:
df[df['is_vip'].isna()].shape

(17112, 17)

In [11]:
df[(df['status_p']=='0')&(df['is_vip'].notna())]['order_id'].nunique()
# df[['create_time', 'order_id']]

3688

# 发货率

In [12]:
df2['下单日期'] = df2.create_time.dt.date

In [13]:
df2_zm = df2[(df2.name.str.contains(r'芝麻|租物'))&(~df2.status.isin([13]))]
df2_zm_fh = df2_zm[(df2_zm.status.isin([3,4]))&(df2_zm.go_express_date-df2_zm.min_create_time<=pd.Timedelta(hours=48))] # 
df2_zm_fh.order_number.nunique()/df2_zm.order_number.nunique()

0.19290123456790123

In [14]:
df2_zm_group = df2_zm.groupby('下单日期').agg(分母=('order_number', 'nunique'))
df2_zm_fh_group = df2_zm_fh.groupby('下单日期').agg(分子=('order_number', 'nunique'))
df2_zm_groups = df2_zm_group.merge(df2_zm_fh_group, on='下单日期', how='left')
df2_zm_groups['发货率'] = (df2_zm_groups.分子/df2_zm_groups.分母).map(lambda x: format(x, '.2%'))
df2_zm_groups.to_excel('F:/需求/瑞彤需求/发货率4.xlsx')